<a href="https://colab.research.google.com/github/whkwls2653/Kaggle_JaneStreet/blob/main/colab_GPU_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
########################################여기 참조하세요!!!!!!!!!!!!!!#####################
tf.test.is_gpu_available()

tf.test.gpu_device_name() 
tf.__version__
!nvidia-smi
########################################여기 참조하세요!!!!!!!!!!!!!!#####################

'2.4.0'

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices
import os

SEED = 1111

tf.random.set_seed(SEED)
np.random.seed(SEED)


save=True
resume=False
########################################여기 참조하세요!!!!!!!!!!!!!!#####################

print("reading csv input .....")
if 'train' in locals():
  print("Alreday have, skip reading")
else:
  train = pd.read_csv('/content/gdrive/MyDrive/Kaggle/janestreet/train.csv')
print("finish reading input")
########################################여기 참조하세요!!!!!!!!!!!!!!#####################



train = train.query('date > 85').reset_index(drop = True) 
train = train[train['weight'] != 0]



train.fillna(train.mean(),inplace=True)



train['action'] = ((train['resp'].values) > 0).astype(int) #* ((train['feature_0'].values)==-1).astype(int)

# train['action'] = (train[featrues[0]==1).astype(int)

features = [c for c in train.columns if "feature" in c]

f_mean = np.mean(train[features[1:]].values,axis=0)

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

X_train = train.loc[:, train.columns.str.contains('feature')]
#y_train = (train.loc[:, 'action'])

y_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T







########################################여기 참조하세요!!!!!!!!!!!!!!#####################
with tf.device('/device:GPU:0'):
  def create_mlp(
      num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
  ):

      inp = tf.keras.layers.Input(shape=(num_columns,))
      x = tf.keras.layers.BatchNormalization()(inp)
      x = tf.keras.layers.Dropout(dropout_rates[0])(x)
      for i in range(len(hidden_units)):
          x = tf.keras.layers.Dense(hidden_units[i])(x)
          x = tf.keras.layers.BatchNormalization()(x)
          x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)

          x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

      x = tf.keras.layers.Dense(num_labels)(x)
      out = tf.keras.layers.Activation("sigmoid")(x)

      model = tf.keras.models.Model(inputs=inp, outputs=out)
      model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
          loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
          metrics=tf.keras.metrics.AUC(name="AUC"),
      )

      return model


  batch_size = 5000
  hidden_units = [145, 145, 145]
  dropout_rates = [0.20, 0.20, 0.20, 0.20]
  label_smoothing = 1e-2
  learning_rate = 1e-3


  checkpoint_path = "/content/gdrive/MyDrive/Kaggle/janestreet/simplemlp_best_save_only/cp.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)
  if save==True:

    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True,monitor='AUC',mode='max', save_best_only=True, verbose=1)





  clf = create_mlp(
      len(features), 5, hidden_units, dropout_rates, label_smoothing, learning_rate
      )

  if resume==True:
    clf.load_weights(checkpoint_path)
  else:
    clf.fit(X_train, y_train, epochs=2000, batch_size=5000,callbacks=[cp_callback] )
########################################여기 참조하세요!!!!!!!!!!!!!!#####################




models = []

models.append(clf)

th = 0.5000


f = np.median
models = models[-3:]
import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = np.mean([model(x_tt, training = False).numpy() for model in models],axis=0)
        pred = f(pred)
        pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)

reading csv input .....
finish reading input
Epoch 1/2000
315/315 [==============================] - 6s 11ms/step - loss: 0.7089 - AUC: 0.5167

Epoch 00001: AUC improved from -inf to 0.52346, saving model to /content/gdrive/MyDrive/Kaggle/janestreet/simplemlp_best_save_only/cp.ckpt
Epoch 2/2000
315/315 [==============================] - 3s 11ms/step - loss: 0.6909 - AUC: 0.5353

Epoch 00002: AUC improved from 0.52346 to 0.53657, saving model to /content/gdrive/MyDrive/Kaggle/janestreet/simplemlp_best_save_only/cp.ckpt
Epoch 3/2000
315/315 [==============================] - 3s 11ms/step - loss: 0.6902 - AUC: 0.5395

Epoch 00003: AUC improved from 0.53657 to 0.54061, saving model to /content/gdrive/MyDrive/Kaggle/janestreet/simplemlp_best_save_only/cp.ckpt
Epoch 4/2000
315/315 [==============================] - 3s 11ms/step - loss: 0.6897 - AUC: 0.5428

Epoch 00004: AUC improved from 0.54061 to 0.54325, saving model to /content/gdrive/MyDrive/Kaggle/janestreet/simplemlp_best_save_only/cp

In [ ]:
a=[[1,2,3],[4,5,6]]

In [ ]:
b=(a==1)
print(b)

True


In [ ]:
print(train)
a=(train['date']==86).astype(int)* (train['weight']<1).astype(int)
print(a)
# b=(train['weight']<1).astype(int)
# print(b)
# print(a * b)

NameError: ignored